In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mgimg
import matplotlib.colors as colors
import scipy as sp
import numpy as np
import nestkit
import corner
import pickle as pickle
from IPython.display import display
%matplotlib inline
import forwardmodel
import ciamod
import TPmod
import cloudnest
import band
import brewtools
import pickup
import settings
from pymultinest.analyse import Analyzer
from astropy.convolution import convolve, convolve_fft
from astropy.convolution import Gaussian1DKernel
from scipy.io.idl import readsav
from scipy import interpolate
from scipy.interpolate import interp1d
from scipy.interpolate import InterpolatedUnivariateSpline
#from bensconv import spex_non_uniform
#from bensconv import conv_uniform_r
#from bensconv import conv_uniform_fwhm

In [ ]:
runname ="G570D_NestTest"

argfile =runname+"_runargs.pic"

runargs = brewtools.pickle_load(argfile)
#with open(argfile, 'rb') as input:
#    runargs = pickle.load(input) 

gases_myP,chemeq,dist,dist_err,cloudtype,do_clouds,gasnum,gaslist,cloudnum,inlinetemps,coarsePress,press,inwavenum,linelist,cia,ciatemps,use_disort,fwhm,obspec,proftype,do_fudge, prof,do_bff,bff_raw,ceTgrid,metscale,coscale = runargs

ndim,pnames = nestkit.countdims(runargs, plist=True)

a = Analyzer(ndim, outputfiles_basename = runname)
stats = a.get_stats()
bestfit_params = a.get_best_fit()
print(bestfit_params)

In [ ]:
theta_max = np.array(bestfit_params['parameters'])
max_like = bestfit_params['log_likelihood']

In [ ]:
tmp = a.get_equal_weighted_posterior()
samples = tmp[:,:(tmp.shape[1]-1)]


In [ ]:
Tsamples = samples[:,ndim-5:]
nsamps = Tsamples.shape[0]
nlayers = len(press)
Tprofs = np.empty([nlayers,Tsamples.shape[0]])
for i in range(0,nsamps):
    Tprofs[:,i] = TPmod.set_prof(1,coarsePress,press,Tsamples[i,:])

In [ ]:
Tlays = np.empty([nlayers,5])
for i in range(0,nlayers):
    junk = Tprofs[i,:]
    junk2 = np.percentile(junk, [2.4,16, 50, 84,97.6],axis=0)
    junk3 = np.array(junk2)
    Tlays[i,:] = junk3[:]
    
#print Tlays
np.array(Tlays[:,1])

In [ ]:
# get the cloud stuff
cloudsamples = samples[:,ndim-9:ndim-5]

cloud_Top = np.percentile(cloudsamples[:, 0], [16, 50, 84])
cloud_height = np.percentile(cloudsamples[:, 1], [16, 50, 84])

In [ ]:
#modP1,modT1 = np.loadtxt("../Saumon_models/cmp/t750g1000nc_m0.0.cmp",skiprows=1,usecols=(1,2),unpack=True)
#modP2,modT2 = np.loadtxt("../Saumon_models/cmp/t750g3160nc_m0.0.cmp",skiprows=1,usecols=(1,2),unpack=True)
#modP3,modT3 = np.loadtxt("../Saumon_models/cmp/t750g316nc_m0.0.cmp",skiprows=1,usecols=(1,2),unpack=True)
#modP3,modT3 = np.loadtxt("../Saumon_models/cmp/t750g316nc_m0.0.cmp",skiprows=1,usecols=(1,2),unpack=True)
#modP4,modT4 = np.loadtxt("G570D_MikeLines_profile.dat",skiprows=0,usecols=(0,1),unpack=True)

In [ ]:
# This may need tweaking - it is untested. 
plt.rc('font',family='Times New Roman')
fig=plt.figure(dpi=320)

prf = fig.add_axes([0.1, 0.1, 0.65, 0.85])

prf.axis([0., 4000.,3.0,-4.0])
plt.ylabel(r'log(P / bar)')
plt.xlabel('T / K')

cld1 = fig.add_axes([0.89,0.1,0.07,0.85],xticklabels=[],xlim=[0,1])
cld1.axis([0,1,3,-4])
cld1.tick_params(axis='y',direction='in',left=True,right=True,labelleft=False,labelright=True)

BesTprof = TPmod.set_prof(proftype,coarsePress,press,theta_max[ndim-5:])

npts = 10

cmap = mpl.cm.Blues
y = np.linspace(cloud_Top[1] - cloud_height[1],cloud_Top[1],npts)
normalize = mpl.colors.Normalize(vmin=y.min(), vmax=y.max()+2)
for j in range(0,npts-1):
        cld1.fill_between([0.,1.],y[j],y[j+1],facecolor=cmap(normalize(y[j])), linewidth=0.0)
        
cld1.fill_between([0.,1.],3.0,cloud_Top[1],facecolor='blue',alpha=0.3)
cld1.fill_between([0,1.],cloud_Top[2],cloud_Top[0],facecolor='grey',alpha=0.8)
cld1.plot([0,1],[cloud_Top[1] -cloud_height[1],cloud_Top[1] - cloud_height[1]],'k--',linewidth=1.0)

#cld3.plot([0,1],[cloud3_Top[2] - cloud3_height[0],cloud3_Top[2] - cloud3_height[0]],'k-.',linewidth=1.0)
#cld3.plot([0,1.],[cloud3_Top[0] - cloud3_height[2],cloud3_Top[0] - cloud3_height[2]],'k-.',linewidth=1.0)

s1 = 'Cld 1'

cld1.text(0.4,2.8,s1,size=8)

logP = np.log10(press)
d1, = prf.plot(Tlays[:,2],logP,'k-')
prf.fill_betweenx(logP,Tlays[:,1], Tlays[:,3], facecolor='red', alpha=0.3)
prf.fill_betweenx(logP,Tlays[:,0], Tlays[:,4], facecolor='red', alpha=0.1)

#d2, = prf.plot(BesTprof,logP,'k-.',linewidth=0.5)


#m1, = prf.plot(modT1,np.log10(modP1),'m-',linewidth=1.2,label='SM 1700K log g = 5.0 f3')
#m2, = prf.plot(modT3,np.log10(modP3),'c-',linewidth=1.2,label='SM 1900K log g = 5.5 f2')
#m3, = prf.plot(modT4,np.log10(modP4),'y-',linewidth=1.2,label='DRIFT 1700K log g = 5.0')
#m4, = prf.plot(modT5,np.log10(modP5),'r-',linewidth=1.2,label='DRIFT 1900K log g = 5.5')

#plt.plot(modT2,np.log10(modP2),'y-',linewidth=2)
#plt.plot(modT3,np.log10(modP3),'y-',linewidth=2)


pero = 10.0**4 / (5.125 - 0.277*logP - 0.554*0.0)

qtz = 10.0**4 / (6.12 - 0.34*logP - 0.69*0.0)
qtz_alt = 10.0**4 / (6.27 - 0.34*logP - 0.69*0.0)


enst = 10.0**4/(6.26 - 0.35*logP-0.70*0.0)
fost = 10.0**4/(5.89 - 0.37*logP-0.73*0.0)
iron = 10.0**4/(5.44 - 0.48*logP-0.48*0.0)
cr =  10.0**4/(6.528 - 0.491*logP-0.491*0.0)
al2o3 = 10.0**4 / (5.0139 - 0.21794*(logP) + 2.2636E-03*(logP)**2.0 - 0.580*0.0)

c1, = prf.plot(enst,logP,'--',color='blue',linewidth=1.5, label='MgSiO$_3$')
c2, = prf.plot(qtz,logP,'--',color='magenta',linewidth=1.5, label='SiO$_2$')
c3, = prf.plot(fost,logP,'--',color='pink',linewidth=1.5,label='Mg$_2$SiO$_4$')
c4, = prf.plot(iron,logP,'--',color='orange',linewidth=1.5, label='Fe')
c5, = prf.plot(pero,logP,'--',color='purple',linewidth=1.5, label='CaTiO$_3$')
c6, = prf.plot(al2o3,logP,'--',color='red',linewidth=1.5, label='Al$_2$O$_3$')

prf.legend(handles=[d1,c1,c2,c3,c4,c5,c6],prop={'size':8}) 


#asp = 10 / 3.5

#plt.axes().set_aspect(asp)
plt.savefig(runname+"_profile.png",format='png', dpi=320)

In [ ]:
np.savetxt(runname+'_profile.dat', np.c_[logP, Tlays[:,2]])

In [ ]:
fig = corner.corner(samples,scale_hist=True,plot_datapoints =False,labels=pnames,quantiles=[0.16, 0.5, 0.84],show_titles=True, title_kwargs={"fontsize": 20},label_kwargs={"fontsize": 20})
plt.savefig(runname+"_fullcorner.png",format='png', dpi=320)

In [ ]:
# uncomment and edit if you want a gas fractions corner plot
#gassamples = samples[:,0:9]
#print(gassamples.shape)
#print(gassamples[0:11,0:11])
#rh2o = (-4,.3)
#rch4 = (-4,-3.)
#rco = (-12.0, 0.)
#rco2 = (-12,0.)
#rnh3= (-5,-4.)
#rh2s = (-12,-0)
#rnak = (-6.,-5)
#rlogg = (1.,80.)
#rrad = (0.5, 2.0)
#bnds = [rh2o,rch4,rco,rco2,rnh3,rh2s,rnak,rlogg,rrad]

#fig = corner.corner(gassamples,scale_hist=False, range = bnds,plot_datapoints =False,labels=['h2o','ch4','co','co2','nh3','h2s',"Na+K","M_jup","Rjup"],quantiles=[0.16, 0.5, 0.84],show_titles=True, title_kwargs={"fontsize": 20},label_kwargs={"fontsize": 20})
#fig = triangle.corner(samples)
#plt.savefig(runname+"_gascorner.png",format='png', dpi=320)


In [ ]:
gasVMR = np.ones([gasnum.size-1,3])
for i in range (0,gasnum.size-1):
    gasVMR[i,:] = np.percentile(samples[:,i], [16, 50, 84])
    
np.savetxt(runname+'_VMRs.dat', np.c_[gasVMR[:,0],gasVMR[:,1],gasVMR[:,2]])


In [ ]:
# get diagnostics along with the spectrum
gnostics = 0
shiftspec, photspec, tauspec,cfunc = nestkit.modelspec(theta_max,runargs,gnostics)

In [ ]:
topspec = brewtools.proc_spec(shiftspec,theta_max,fwhm,chemeq,gasnum,obspec) 
print(topspec.shape)

In [ ]:
pltspec = np.zeros((5000,obspec[0,:].size))
samp= np.empty(ndim)
sid = np.zeros(5000)
for i in range (0,5000):
    sid[i]= np.random.randint(0,high = len(samples))
    samp = samples[int(sid[i]),:]
    shiftspec, photspec, tauspec,cfunc = nestkit.modelspec(samp,runargs,gnostics)
    pltspec[i,:] = brewtools.proc_spec(shiftspec,samp,fwhm,chemeq,gasnum,obspec) 

In [ ]:
specdist = np.empty([obspec[0].size,5])
for i in range(0,obspec[0].size):
    junk = pltspec[:,i]
    junk2 = np.percentile(junk, [2.4,16, 50, 84,97.6],axis=0)
    junk3 = np.array(junk2)
    specdist[i,:] = junk3[:]

In [ ]:
plt.rc('font',family='Times New Roman')
fig=plt.figure(dpi=320)
plt.axis([1.0,2.5,0,1e-14])
#plt.axis([0.6,15.0, 1.e-17,150e-15])

#ax = fig.add_subplot(1,1,1)

#ax.set_yscale('log')
#ax.set_xscale('log')

for i in range(0,5000):
    plt.plot(obspec[0],pltspec[i,:],color='r',alpha=0.01)


r1, = plt.plot(obspec[0],specdist[:,2],'y-',linewidth=0.5, label = "median")
#plt.fill_between(obspec[0],specdist[:,0],specdist[:,4],facecolor='red',alpha=0.2)
plt.fill_between(obspec[0],specdist[:,1],specdist[:,3],facecolor='red',alpha=0.5)

t1, = plt.plot(obspec[0,:],topspec,'g-',linewidth=1, label = "max likelihood")
d1, = plt.plot(obspec[0,:],obspec[1,:],'k-',label = runname+" data")
#plt.fill_between(obspec[0,:],obspec[1,:]-obspec[2,:],obspec[1,:]+obspec[2,:],facecolor='red',alpha=0.2)

r1, = plt.plot(obspec[0],specdist[:,2],'y-',linewidth=0.5, label = "median")
#plt.fill_between(obspec[0],specdist[:,0],specdist[:,4],facecolor='red',alpha=0.2)
#plt.fill_between(obspec[0],specdist[:,1],specdist[:,3],facecolor='red',alpha=0.5)

#m1, = plt.plot(markwave,modspec,'m-', label="SM 1700K",linewidth=0.7)
#m2, = plt.plot(markwave2,modspec16,'c-', label="SM 1600K",linewidth=0.7)
#d2, = plt.plot(longspec[0,:],longspec[1,:],'k-')

#plt.plot(isow1,isoflux1,'ro')
#plt.errorbar(isow1,isoflux1,yerr=isofluxerr1)
#plt.plot(isow2,isoflux2,'ro')
#plt.errorbar(isow2,isoflux2,yerr=isofluxerr2)
#plt.plot(isow3,isoflux3,'ro')
#plt.errorbar(isow3,isoflux3,yerr=isofluxerr3)


#plt.plot([2.4,2.4],[0,1.0],'k--')
#plt.plot([0.8,0.8],[0,1.0],'k--')

plt.legend(handles=[d1,t1])


plt.ylabel(r'$ F_{\lambda}$ / $Wm^{-2} \mu m^{-1}$')
plt.xlabel('Wavelength / $\mu m$')
#plt.savefig(runname+"_SPAG_SPEC.png",format='png', dpi=320)

In [ ]:
#np.savetxt(runname+'_MEDIAN_SPEC.dat', np.c_[obspec[0,:],specdist[:,2],specdist[:,1],specdist[:,3]])

In [ ]:
#np.savetxt(runname+'_MAX_LIKE_SPEC.dat', np.c_[obspec[0,:],topspec])

In [ ]:
# grab BFF and Chemical grids
print(gaslist)
chemeq=1
bff_raw,ceTgrid,metscale,coscale,gases_myP = nestkit.sort_bff_and_CE(chemeq,"chem_eq_tables_P3K.pic",press,gaslist)
   
# what metallicity and C/O ratio do we want to plot?
mh  = -0.00
co = (0.55/0.55)

logP, profT = np.loadtxt(runname+"_profile.dat",unpack=True)

nlayers = press.size
mfit = interp1d(metscale,gases_myP,axis=0)
gases_myM = mfit(mh)
cfit = interp1d(coscale,gases_myM,axis=0)
invmr = cfit(co)
ng = invmr.shape[2]
ngas = ng - 3
ab = np.zeros([nlayers,ngas],dtype='d')
temp= profT #Tlays[:,2]
for p in range(0,nlayers):
    for g in range(3,ng):
        tfit = InterpolatedUnivariateSpline(ceTgrid,invmr[:,p,g])
        ab[p,g-3]= tfit(temp[p])

vmr = gasVMR

plt.rc('font',family='Times New Roman')
plt.rc('text', usetex=True)
fig=plt.figure(dpi=320)
fig.set_size_inches(6., 6.)
fig, axs = plt.subplots(ngas-1,sharex='col', sharey='row',gridspec_kw={'hspace': 0, 'wspace': 0})


asp = 0.5

for i in range(0,ngas-2):
    lsize = 8
    # row and column sharing

    axs[i].set(xlim=[-12, 0], ylim=[2.5,-4], aspect=asp)
    axs[i].plot(ab[:,i],logP,'--',color='red',linewidth=1.5)
    c1, = axs[i].plot([vmr[i,1],vmr[i,1]],[2.4,-4.0],'-',color='red',label=gaslist[i].upper())
    axs[i].fill_betweenx([2.4,-4.0],[vmr[i,0],vmr[i,0]],[vmr[i,2],vmr[i,2]], facecolor='red', alpha=0.1,linewidth=0)
    axs[i].legend(handles=[c1],prop={'size':lsize})

# this bit is Na+K tied together. If they're not, remove this bit, and run the loop above to ngas
alks = np.log10((10.**ab[:,ngas-2]) + (10.**ab[:,ngas-1]))

axs[ngas-2].set(xlim=[-12, 0], ylim=[2.5,-4], aspect=asp)  
axs[ngas-2].tick_params(axis='y',direction='in',left=True,right=True)
axs[ngas-2].plot(alks,logP,'--',color='red',linewidth=2)
c7, = axs[ngas-2].plot([vmr[ngas-2,1],vmr[ngas-2,1]],[2.4,-4.0],'-',color='red',label='Na+K')
axs[ngas-2].fill_betweenx([2.4,-4.0],[vmr[ngas-2,0],vmr[ngas-2,0]],[vmr[ngas-2,2],vmr[ngas-2,2]], facecolor='red', alpha=0.1,linewidth=0)
axs[ngas-2].legend(handles=[c7],prop={'size':lsize})



fig.txt = fig.text(0.5, 0.04, '$\log f_{gas}$', ha='center')
fig.text(0.3, 0.5, '$\log (P / bar)$', va='center', rotation='vertical')
plt.savefig(runname+'_abundances.png',format='png', dpi=320)

In [ ]:
# get diagnostics along with the spectrum
gnostics = 1
shiftspec, clphotspec, ophotspec,cfunc = nestkit.modelspec(theta_max,runargs,gnostics)

nwave = inwavenum.size
cfunc = np.reshape(cfunc,[cfunc.shape[1],cfunc.shape[2]])
fwhm = 0.005
wlen = shiftspec.shape[1]
wint =  shiftspec[0,0] - shiftspec[0,wlen-1]
# convolve with instrumental profile
# start by setting up kernel
# First step is finding the array index length of the FWHM
disp = wint / wlen
gwidth = int((((fwhm / disp) // 2) * 2) +1)
# needs to be odd
# now get the kernel and convolve
gauss = Gaussian1DKernel(gwidth)

for ilayer in range (0,press.size):
    cfunc[:,ilayer] = convolve(cfunc[:,ilayer],gauss,boundary='extend')

tau1_cl_Press = convolve(clphotspec[0],gauss,boundary='extend')[::-1]
tau1_oth_Press = convolve(ophotspec[0],gauss,boundary='extend')[::-1]
    
wavenew = shiftspec[0,::-1]
press = press.reshape(64,)
normfunc = np.zeros_like(cfunc)
for iwave in range(0,nwave):
    totcont = np.sum(cfunc[iwave,:])
    normfunc[iwave,:] = cfunc[iwave,:] / totcont

    
plt.rc('font', family='serif')
plt.rc('text', usetex=False)
fig=plt.figure(dpi=120)
plt.axis([wavenew[0],wavenew[-1],press[-1],press[0]])

ax = fig.add_subplot(1,1,1)

ax.set_yscale('log')
ax.set_xscale('log')
#major_ticks = np.arange(1.0,15.,1.0)
#minor_ticks = np.arange(1.0,15.,0.5)
#ax.set_xticks(major_ticks)                                                       
#ax.set_xticks(minor_ticks, minor=True)                                           

plt.pcolormesh(wavenew,press,(normfunc[::-1,:].transpose()),cmap='Greys',norm=colors.SymLogNorm(linthresh=0.001,linscale=0.00001,
                                              vmin=0., vmax=np.amax(normfunc)))

t1, = plt.plot(wavenew,(tau1_cl_Press),'m-',label=r'$\tau_{cloud} = 1.0$')
t2, = plt.plot(wavenew,(tau1_oth_Press),'c-', label =r'$\tau_{gas} = 1.0$')

plt.legend(handles=[t2])

cbar = plt.colorbar(orientation='vertical',norm=colors.Normalize(clip=False),ticks=[1e-3,1e-2,0.1])
cbar.ax.set_yticklabels(['<0.1%', '1%', '10%'])
cbar.set_label('% of total', rotation=270)
plt.ylabel('Pressure / bar)')
plt.xlabel('Wavelength / $\mu m$')
plt.savefig(runname+'_contribution.png',format='png', dpi=120)